<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

**Выделение названий населённых пунктов из представленных данных. Названия выделяются с помощью библиотеки Natasha и добавляются в отдельный столбец**

In [1]:
import pandas as pd
import re
import natasha
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc)

import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [2]:
df=pd.read_csv(r'.......csv',sep=';')
df.head()

,ID автора,Ссылка на автора,Имя и фамилия автора,Пол автора,Аватара автора,Ссылка на комментарий,Дата и время,Текст комментария,Число лайков к комментарию
0,15944547,https://vk.com/id15944547,Вадим Маклаков,М,https://sun1-30.userapi.com/s/v1/ig2/25EPoH0Uk...,https://vk.com/wall-87598739_763439?reply=763441,10 июля 2022 09:23,Все ещё не очень (мос обл ) злаковые,1
1,385187645,https://vk.com/id385187645,Лилия Таршикова,Ж,https://vk.com/images/camera_100.png,https://vk.com/wall-87598739_763439?reply=763444,10 июля 2022 10:27,Доброе утро. Московская область. Балашиха. Все...,0
2,579547704,https://vk.com/id579547704,Анастасия Некрасова,Ж,https://sun1-19.userapi.com/s/v1/ig2/SkfND4ulA...,https://vk.com/wall-87598739_763439?reply=763450,10 июля 2022 10:44,Вчера и сегодня ребенок 3 года. Очень плохо ды...,0
3,720361644,https://vk.com/id720361644,Яна Николаенко,Ж,https://sun1-84.userapi.com/s/v1/ig2/3xR1rej7J...,https://vk.com/wall-87598739_763439?reply=763456,10 июля 2022 11:04,"[id579547704|Анастасия], Что у вас сейчас пыли...",0
4,579547704,https://vk.com/id579547704,Анастасия Некрасова,Ж,https://sun1-19.userapi.com/s/v1/ig2/SkfND4ulA...,https://vk.com/wall-87598739_763439?reply=763505,10 июля 2022 14:08,Вроде нет,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16264 entries, 0 to 16263
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID автора                   16264 non-null  int64 
 1   Ссылка на автора            16264 non-null  object
 2   Имя и фамилия автора        16256 non-null  object
 3   Пол автора                  16242 non-null  object
 4   Аватара автора              16256 non-null  object
 5   Ссылка на комментарий       16264 non-null  object
 6   Дата и время                16264 non-null  object
 7   Текст комментария           16210 non-null  object
 8   Число лайков к комментарию  16264 non-null  int64 
dtypes: int64(2), object(7)
memory usage: 1.1+ MB


In [4]:
df['Текст комментария']=df['Текст комментария'].fillna('unknow')

In [5]:
df['Текст комментария'].dropna(inplace=True)
df['tokens'] = df['Текст комментария'].apply(word_tokenize)
df['tokens'].head(20)

0       [Все, ещё, не, очень, (, мос, обл, ), злаковые]
1     [Доброе, утро, ., Московская, область, ., Бала...
2     [Вчера, и, сегодня, ребенок, 3, года, ., Очень...
3     [[, id579547704|Анастасия, ], ,, Что, у, вас, ...
4                                          [Вроде, нет]
5     [Добрый, день, ., Самочувствие, плохое, ,, оче...
6     [Москва, ,, проснулся, с, кашлем, и, хрипами, ...
7                      [Мытищи, ., Почти, нормально, .]
8     [СПб, ., Чихание, и, небольшой, отек, в, носу,...
9     [Спб, ., Ребёнок, без, АГ, пару, дней, ., Само...
10    [Вроде, дело, к, концу, пошло, ., Можно, вздох...
11    [У, меня, тоже, к, концу, идёт, обычно, к, 10,...
12    [Вчера, днём, покосила, на, даче, в, очках, ,,...
13    [Вологодская, область, ., Сегодня, проснулась,...
14    [Эх, ,, когда-то, я, живьем, видела, это, озер...
15                [Какая, красота😍, хочу, туда, !, 🤗🤗🤗]
16                                                  [😍]
17    [После, ливня, сегодня, хуже, ,, это, я, т

In [6]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
addr_extractor = AddrExtractor(morph_vocab)
def loc (row): 
    for i in row['tokens']:
                
        doc = Doc(i)
        doc.segment(segmenter)
        #doc.tokens
        doc.tag_morph(morph_tagger)
        doc.parse_syntax(syntax_parser)
        doc.tag_ner(ner_tagger)
        for span in doc.spans:
            span.normalize(morph_vocab)
            for span in doc.spans:
                if span.type == 'LOC':
                    #span.extract_fact(addr_extractor)
                    return span.text
        for match in addr_extractor(i):
            return match.fact.value


#Из данных выделяется число как адрес. Его у меня не получилось убрать ни с помощью регулярного выражения, ни просто руками                   
                
        
     

In [7]:
df['loc']=df.apply(loc, axis=1)

In [8]:
df[['loc','Текст комментария']].head(30)

,loc,Текст комментария
0,None,Все ещё не очень (мос обл ) злаковые
1,Балашиха,Доброе утро. Московская область. Балашиха. Все...
2,Курск,Вчера и сегодня ребенок 3 года. Очень плохо ды...
3,None,"[id579547704|Анастасия], Что у вас сейчас пыли..."
4,None,Вроде нет
5,Москва,"Добрый день. Самочувствие плохое, очень сильны..."
6,Москва,"Москва, проснулся с кашлем и хрипами, запил АГ..."
7,Мытищи,Мытищи. Почти нормально.
8,СПб,СПб. Чихание и небольшой отек в носу. Аллергия...
9,Спб,Спб. Ребёнок без АГ пару дней. Самочувствие хо...


In [9]:
df.to_csv('vk_location1.csv',index=False)